In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 
set_to_release = ['eae5ee86-b6ee-4112-941b-311d6026acb7']
store, uuids = record_object_es(set_to_release, my_auth, schema_name, store_frame='raw', add_pc_wfr=True, store={}, item_uuids=[])
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1)/60)
   

GET http://search-fourfront-webprod-hmrrlalm4ifyhl4bzbvl73hwv4.us-east-1.es.amazonaws.com:80/_all/_search [status:400 request:0.045s]


RequestError: TransportError(400, 'parsing_exception', '[terms] query does not support [_id]')

In [ ]:

def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
 #   add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, key=my_auth, frame='object')['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, key=my_auth, frame = 'raw')
    
    if resp['status'] not in ['current', 'released']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, key=my_auth, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key in ['uuid', 'blob_id', "attachment"]:
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
  
pub = 'b8aada45-1504-428d-91b5-355bc9d158af'
store_dict = {} 
item_uuids = []

store, uuids = record_object(pub, store_dict, item_uuids)

print(len(uuids))
print(len(list(set(uuids))))
    


In [ ]:
import os

folder = 'Json'

if not os.path.exists(folder):
    os.makedirs(folder)

for a_type in store:
    coll = store[a_type]
    for a_ele in coll:
        print a_type, a_ele['status']
#         #
#         for avoid in ['attachment', 'last_modified']:
#             if avoid in a_ele.keys():
#                 del a_ele[avoid]

# for a_type in store:
#     filename = folder + '/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)

In [ ]:
## 

all_uuids = ["0078b155-31ca-4dec-86f0-4bd2ee54e4f9",
      "023c0d3f-8f46-447a-a712-335ac5171444",
      "032cb6e7-02e5-4713-9ba0-1ab4e6af9360",
      "054197e4-e715-404a-85ac-713a80209ce8",
      "0ba23514-445b-4f66-a8db-99693524f498",
      "0c22303f-2fe9-4556-866a-0b5db91fd814",
      "104d2869-6d79-4ec1-8c89-cc02c3833b80",
      "111202bc-8535-4448-903e-854af460a233",
      "15f71f52-eef7-43b6-86ca-52f78dc6f6a2",
      "17b3f532-060e-409c-85d2-0ddda423fcc4",
      "185be935-a30d-44e6-af70-ddea9f3d061b",
      "1d12a957-c3f7-4854-b390-187e29180bce",
      "2233a2d3-8e71-4656-ba32-6d2eb5db8895",
      "289a75d0-a59c-47a4-b4cb-ff2fd62e6bac",
      "2983c9ec-e793-4ce6-8b10-309ebebb17b3",
      "2da280f1-236c-42cd-9066-cb699f59bb33",
      "2fd4da21-5db2-40a6-a20e-2f425b823114",
      "344db0af-078b-42ea-ae5c-c417194e36eb",
      "376abbf7-7ebb-4ec2-8710-d14a4759559b",
      "37b9c26c-52f8-4cfa-98e9-cd830ecaa03c",
      "37de39a5-869d-4da0-a9e0-ae9596c11020",
      "3f640a02-d8e4-420f-b8ab-9bc42312748e",
      "4042d90e-9805-4819-bb05-6bf2ce3eda05",
      "43f75f61-075b-4bc3-88ea-5d37e56f8a8e",
      "44dcd824-2956-40c3-8425-85798f05ccd3",
      "46a49b5d-f185-4e98-8747-c31a904bbc2f",
      "4871e338-b07d-4665-a00a-357648e5bad6",
      "48a5ee68-e84b-4da2-9ecb-fe1736f98a9e",
      "4a3044c1-8b05-47a4-ae73-49cf9aa59053",
      "4a621a38-1df8-4d87-b414-25d74dacaec7",
      "500ddfca-6f74-4b20-a470-b34b46c23c24",
      "54a124db-abe4-4525-a0c5-750755486bd4",
      "54ffd3fb-8da9-4fbf-8d32-45c631d68193",
      "5742cf29-7b84-4bee-a34d-3474bfe905b4",
      "5b480cb9-a80d-477b-a802-31077e25a993",
      "5b5f17be-a7b5-4c60-a1f3-37fd4d31b6a2",
      "5dea39d8-4fb8-44d8-a3ea-f947e7010ebe",
      "60c1c426-3d5f-418b-8ac2-f6e99a3d046a",
      "611a9b95-8d81-40bc-9a16-1d13c01708fe",
      "675e58df-1263-4595-a35b-5b19790730e4",
      "69b9af01-d743-4ce1-ba5e-e17b94e918ba",
      "6f052613-bdb4-4349-8444-0a743051b3d2",
      "6f56549d-d916-4d6b-8656-3ed408840f7d",
      "6f656f82-6d5e-4f7e-b409-d4a7c37e5e78",
      "759ea3f7-59e1-4811-93b1-ab875335a771",
      "77da562e-d0ec-45a8-8764-14fa16d995a1",
      "77ec51e4-1d4b-4718-b9b3-a6ccefde43af",
      "791f2e4a-0bbe-40ad-bb7f-892f473f05e0",
      "795847de-20b6-4f8c-ba8d-185215469cbf",
      "79669e61-ceb0-4aa4-9cfa-0fa64a3d1acd",
      "7a32bd32-c447-44f3-8ea8-4ef76c04c9bc",
      "7f29279f-cd8b-4fc7-b985-4e86f1702276",
      "80e1c5cd-8ba1-4511-8e66-02a683750ddb",
      "828cd4fe-ebb0-4b36-a94a-d2e3a36cc989",
      "83d2928e-2d7f-468b-b618-94c81722d2f1",
      "83f36c27-c222-4745-8630-e1514aace964",
      "897ed3de-38ff-485f-829f-1336ec6ac37f",
      "8b2107ba-ce86-4115-8a6d-b60d0dce7eae",
      "8c2a10e8-4cdb-4b8c-937d-468699ce3533",
      "8c380474-12cd-41f4-b86f-2a29faf12eec",
      "8ed69ed6-724b-48b7-8ee2-66b471ff90a5",
      "9127e693-bf95-4811-90d3-93e3e0afb785",
      "93df02c6-1047-43f6-afae-d3598b61eeea",
      "944c7588-c8c6-4c6b-a306-550452979661",
      "95e9d330-f285-4a0c-9fd6-9ebd383877d1",
      "967e7342-6295-41ca-89f3-f6bad9f57bea",
      "98306c0d-fd1d-479b-b2d7-894c4d5d0ad4",
      "986b362f-4eb6-4a9c-8173-3ab267307e3a",
      "98a52fc0-6344-42ef-aa14-202d2aabc624",
      "9ad51f65-13f2-452a-bd36-4ff1aa9027c4",
      "9ee1902f-d9fc-48eb-b7b6-654e12c13a7b",
      "9f65c034-4dd4-4015-9a0d-b7e0a7632e1e",
      "9ff939d4-83ae-40e0-a73f-e3f7d572f6c2",
      "aa5b5dcd-ccca-4501-b0a7-486b9057067e",
      "ad14b956-84d9-4ff8-8012-7996e1bbf38c",
      "ad361e83-43cd-4377-84a2-7940dd15c487",
      "af788829-5066-4c95-9bf4-b1fae4c1e952",
      "b0b9c607-f8b4-4f02-93f4-9895b461334b",
      "b25aca10-09c6-4c7f-8ee3-262b0c5116f9",
      "b2cfe4dd-561c-4ac8-96da-16dcdb569f6b",
      "b58ac18a-701f-4774-b37a-b549c6841f6b",
      "b738731d-1429-40f4-8b3a-aef0c7b762e3",
      "b7c73963-ae1b-4032-876c-7b442d42d31d",
      "b89d34d4-124e-44d8-b2a4-fe092e6c6010",
      "b8aada45-1504-428d-91b5-355bc9d158af",
      "b914b199-66e0-4762-a21e-bacf4669d099",
      "ba4a406b-9ba2-4018-b2f7-5f98ba5c3528",
      "bd5d8f6d-ce55-40c6-bebc-549a8727f0e8",
      "bdd26c42-52b2-4f69-aca1-977eab857c05",
      "be9d5167-dcad-45e8-a3d5-a5fd07d349d1",
      "bfac67d1-a68b-422d-863c-303e1f609b02",
      "c3970447-faef-40d3-a010-990ea55c5e7c",
      "c42e3984-ece0-4b57-8d2e-f5c007fdb3f5",
      "c490c874-9565-48c4-8701-1568f328ba46",
      "c6159041-a5d2-4a7c-af80-a0ff973ce149",
      "c774c350-8333-4e5c-99fc-caafee8d5d58",
      "c885c6d4-f25d-492c-92e3-6291d9dfbc60",
      "c8952d74-141f-4e76-bd5f-cec25009e7ea",
      "c8961d71-0935-4c63-a67f-e6048c7e1db3",
      "c9a53ded-ea41-45d0-8564-9b2f87ce9264",
      "c9eec19e-aa86-4156-9fb8-50d146b0ff0d",
      "ca25eaab-82a1-41b9-8fa6-c18a18f44625",
      "cf659aca-176f-4384-bb63-0dafeed61ebb",
      "d29d8756-e8d5-453c-8b2e-a8ac223b8574",
      "d2b37f10-7acd-4d79-8be0-be53616073c6",
      "d34b2505-bd84-45dd-9df7-f6340ec28b56",
      "d41df190-7eb7-493e-8f2b-1cc7affcb2a8",
      "d726a3a4-5332-417a-b0a5-1764fc1d7fa2",
      "d8da42d0-bc20-40e5-bacf-c263b8603e03",
      "d95cfeda-cb01-432d-addd-c7d60915ffe5",
      "da4f53e5-4e54-4ae7-ad75-ba47316a8bfa",
      "daf47375-8964-450b-9392-cb2f043cae59",
      "dc61dac6-14ae-46e5-8482-1323d32ee00c",
      "e0ff3dd1-0bc7-4c10-9976-95bbbe627277",
      "e2b940fc-75da-44cb-a244-8b1d6200f752",
      "e3159ffc-a5a9-43a1-8cfa-90b776c39788",
      "e6997d1f-f8e9-49ff-ac3a-f565a10e52de",
      "e8bc8ae4-3f2d-464a-96b6-91388c4b7bd5",
      "ea4081f7-83e2-4f2f-b18b-c1566d1595db",
      "ee5089e9-53b2-4b80-8b73-0f3049e57780",
      "ef0eba47-ada0-42bc-9329-5145b57ff33a",
      "f048f178-131e-49f3-a393-5e265c90fa2d",
      "f127b496-8f8b-4aab-a4c9-6e24d516a15e",
      "f1d42039-fa49-402d-8b6c-f036919aabff",
      "f3a54565-7287-4084-aae5-c6342bb9be45",
      "f3c74393-ddc7-4a15-9db0-eb9825db4a63",
      "f4e5c8cd-6d39-4a4b-b79f-211795997739",
      "f5fc400a-4d7f-4916-99a9-ffbc82132efe",
      "f8ddbc70-a311-490b-b0ef-7549868a059c",
      "f99ccef2-9ce9-46ea-bfed-d58d4e38b949",
      "fb19f661-edca-4405-a6b3-f9b74f1a95ad",
      "fd7adaab-ac79-4e20-abe3-14dcaef0e105"]

In [ ]:
diff = set(item_uuids) - set(all_uuids)
print list(diff)
print len(list(diff))